# Tutorial Notebook

Ok this is supposed to be a tutorial python notebook to import and use functions from be-scan but I can't figure out the syntax of how to import, and it's not a priority

In [6]:
# control, import pandas
import pandas as pd


In [9]:
from be_scan import plot_boxes


ImportError: attempted relative import with no known parent package